In [1]:
    '''
import tools
    '''
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")
import yfinance as yf
from itertools import product  

In [38]:
class MA_Strategy():
   

    '''
    Class for the backtesting of SMA strategies.
    '''        
    def __init__(self, ticker_name, MA_Short, MA_Long, start, end):
        '''
        Input
        ----------
        ticker_name: str
            ticker symbol to be backtested
        MA_Short: int
            Sliding window in bars (e.g. days) for shorter Simple Moving Average
        MA_Long: int
            Sliding window in bars (e.g. days) for longer Simple Moving Average
        start: str
            start date for data import
        end: str
            end date for data import
        ''' 
        self.ticker_name = ticker_name
        self.MA_Short = MA_Short
        self.MA_Long = MA_Long
        self.start = start
        self.end = end
        self.result = None 
        self.get_data()
        self.prepare_data()
        
    def __repr__(self):
        return "MA_Strategy(ticker = {}, SMA_S = {}, SMA_L = {}, start = {}, end = {})".format(self.ticker_name, self.MA_Short, self.MA_Long, self.start, self.end)
        
    def get_data(self):
        ''' Imports the data from yfinance (source can be changed).
        '''
        StockID = self.ticker_name
        tem = yf.Ticker(StockID+'.TW').history('max').ffill()
        if tem.empty:
                tem = yf.Ticker(StockID+'.TWO').history('max').ffill()
        if tem.empty:
                tem = yf.Ticker(StockID).history('max').ffill()
        df=tem.loc[ self.start: self.end]
        raw = df[["Close"]]
        raw.rename(columns={"Close": "price"}, inplace=True)
        raw["returns"] = np.log(raw / raw.shift(1))
        self.data = raw
        
    def prepare_data(self):
        '''Prepares the data for strategy backtesting (strategy-specific).
        '''
        data = self.data.copy()
        data["SMA_S"] = data["price"].rolling(self.MA_Short).mean()
        data["SMA_L"] = data["price"].rolling(self.MA_Long).mean()
        self.data = data
        
    def set_parameters(self, SMA_S = None, SMA_L = None):
        ''' Updates SMA parameters and the prepared dataset.
        '''
        if SMA_S is not None:
            self.MA_Short = SMA_S
            self.data["SMA_S"] = self.data["price"].rolling(self.MA_Short).mean()
        if SMA_L is not None:
            self.MA_Long = SMA_L
            self.data["SMA_L"] = self.data["price"].rolling(self.MA_Long).mean()
            
    def test_strategy(self):
        ''' Backtests the SMA-based trading strategy.
        '''
        data = self.data.copy().dropna()
        data["position"] = np.where(data["SMA_S"] > data["SMA_L"], 1, -1)
        data["strategy"] = data["position"].shift(1) * data["returns"]
        data.dropna(inplace=True)
        data["creturns"] = data["returns"].cumsum().apply(np.exp)
        data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
        self.results = data
       
        Str_perf = data["cstrategy"].iloc[-1] # absolute performance of the strategy
        alpha_outperf = Str_perf - data["creturns"].iloc[-1] # out-/underperformance of strategy
        print("strategy performance:{}".format(round(Str_perf, 6)), "bit buy and hold:{}".format(round(alpha_outperf, 6)))
        return round(Str_perf, 6),round(alpha_outperf, 6)
    
    def plot_results(self):
        ''' Plots the performance of the trading strategy and compares to "buy and hold".
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        else:
            title = "{} | SMA_S = {} | SMA_L = {}".format(self.ticker_name,self.MA_Short, self.MA_Long)
            self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))
    
    def optimize_parameters(self, SMA_S_range, SMA_L_range):
        ''' Finds the optimal strategy (global maximum) given the SMA parameter ranges.

        Parameters
        ----------
        SMA_S_range, SMA_L_range: tuple
            tuples of the form (start, end, step size)
        '''
        combinations_test = list(product(range(*SMA_S_range), range(*SMA_L_range)))
        
        # test all combinations
        results = []
        for comb in combinations_test:
            self.set_parameters(comb[0], comb[1])
            results.append(self.test_strategy()[0])
        
        best_perf = np.max(results) # best performance
        optimiz = combinations_test[np.argmax(results)] # optimal parameters
        
        # run/set the optimal strategy
        self.set_parameters(optimiz[0], optimiz[1])
        self.test_strategy()
                   
        # create a df with many results
        many_results =  pd.DataFrame(data = combinations_test, columns = ["SMA_S", "SMA_L"])
        many_results["performance"] = results
        self.results_overview = many_results
                            
        return print("optimizer located: {} ".format(optimiz),"optimizer index: {} ".format(best_perf))